<a href="https://colab.research.google.com/github/AndreYang333/LLMdemo/blob/main/FinalAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ProsusAI/finBERT.git

Cloning into 'finBERT'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 168 (delta 6), reused 10 (delta 4), pack-reused 149 (from 1)
Receiving objects: 100% (168/168), 73.57 KiB | 538.00 KiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
!pip install transformers
!pip install torch

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [94]:
import nltk
import pandas as pd
nltk.download('punkt')

import matplotlib.pyplot as plt
import seaborn as sns


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [89]:
def predict(text, model, write_to_csv=False, path=None, use_gpu=False, gpu_name='cuda:0', batch_size=1):
    """
    Predict sentiments of sentences in a given text. The function first tokenizes sentences, make predictions and write
    results.
    Parameters
    ----------
    text: string
        text to be analyzed
    model: BertForSequenceClassification
        path to the classifier model
    write_to_csv (optional): bool
    path (optional): string
        path to write the string
    use_gpu: (optional): bool
        enables inference on GPU
    gpu_name: (optional): string
        multi-gpu support: allows specifying which gpu to use
    batch_size: (optional): int
        size of batching chunks
    """
    model.eval()

    sentences = sent_tokenize(text)
    device = gpu_name if use_gpu and torch.cuda.is_available() else "cpu"
    label_list = ['positive', 'negative', 'neutral']
    label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}
    result = pd.DataFrame(columns=['sentence', 'logit', 'prediction', 'sentiment_score'])
    attention_matrices = []
    for batch in chunks(sentences, batch_size):
        examples = [InputExample(str(i), sentence) for i, sentence in enumerate(batch)]
        features = convert_examples_to_features(examples, label_list, 64, tokenizer)
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long).to(device)
        all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long).to(device)
        all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long).to(device)
        with torch.no_grad():
            model     = model.to(device)

            outputs = model(
                all_input_ids,
                attention_mask=all_attention_mask,
                token_type_ids=all_token_type_ids,
                output_attentions=True
            )
            logits = outputs[0]  # The logits output
            attentions = outputs[-1]  # Attention matrices
            logging.info(logits)
            logits = softmax(np.array(logits.cpu()))
            sentiment_score = pd.Series(logits[:, 0] - logits[:, 1])
            predictions = np.squeeze(np.argmax(logits, axis=1))

            batch_result = {'sentence': batch,
                            'logit': list(logits),
                            'prediction': predictions,
                            'sentiment_score': sentiment_score,
                            }

            batch_result = pd.DataFrame(batch_result)
            result = pd.concat([result, batch_result], ignore_index=True)
            attention_matrices.append([attentions])
    result['prediction'] = result.prediction.apply(lambda x: label_dict[x])

    return result,attention_matrices

In [90]:
text ="""Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.

Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.

There is "way more demand than is even available, so that’s good," said the CEO of Euronext Amsterdam, Maurice van Tilburg. "It’s going to be an interesting hour of trade after opening this morning."

Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).

The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."""

In [91]:
result,attention_matrices = predict(text,model)

<ipython-input-89-eef10a0e8c34>:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, batch_result], ignore_index=True)


In [92]:
len(attention_matrices)

7

In [93]:
result

,sentence,logit,prediction,sentiment_score
0,Shares in the spin-off of South African e-comm...,"[0.94383955, 0.021706667, 0.03445379]",positive,0.922133
1,"Bob van Dijk, CEO of Naspers and Prosus Group ...","[0.027972123, 0.021268595, 0.9507593]",neutral,0.006704
2,REUTERS/Piroschka van de Wouw\nProsus comprise...,"[0.14918718, 0.0077901655, 0.84302264]",neutral,0.141397
3,"There is ""way more demand than is even availab...","[0.9128226, 0.013744192, 0.073433094]",positive,0.899078
4,"""It’s going to be an interesting hour of trade...","[0.7504015, 0.023193007, 0.2264054]",positive,0.727208
5,Euronext had given an indicative price of 58.7...,"[0.08340584, 0.014246288, 0.9023479]",neutral,0.069160
6,The shares jumped to 76 euros on opening and w...,"[0.23598625, 0.017281935, 0.7467318]",neutral,0.218704
